# Gemini AI: Extracción de datos estructurados con Gemini Pro

Este notebook contiene un ejemplo práctico que demuestra cómo realizar búsquedas en Google en tiempo real y procesar los datos obtenidos mediante técnicas avanzadas de web scraping y la aplicación de modelos de inteligencia artificial como Gemini Pro de Vertex AI. Es una guía paso a paso que cubre desde la autenticación y configuración inicial hasta la extracción y formateo de datos.

**CONTENIDO**

- Autenticación en Google Cloud: Código para autenticar el usuario en Google Colab y configurar Vertex AI con un Project ID y una ubicación específica.
- Web Scraping: Un script de Python para extraer contenido de una página web utilizando requests y html2text.
- Procesamiento de Datos con Gemini Pro: Uso del modelo gemini-pro de Vertex AI para formatear la respuesta del scraping web a un esquema JSON predefinido.

**CÓMO EMPEZAR**

- Autenticación y Configuración: Sigue las instrucciones para autenticarte y configurar tu entorno de Vertex AI.
- Extracción de Datos: Realiza web scraping de una URL específica para recopilar datos.
- Formateo de Datos con IA: Utiliza el modelo Gemini Pro para transformar los datos extraídos en un formato estructurado y fácilmente analizable.

**CONTRIBUIR**

Si tienes ideas, preguntas o deseas discutir sobre las posibilidades de la IA y cómo trabajar juntos para construir soluciones basadas en IAG, no dudes en contactarme:

- GitHub: https://github.com/albertgilopez
- LinkedIn: Albert Gil López: https://www.linkedin.com/in/albertgilopez/
- Inteligencia Artificial Generativa (IAG) en español: https://www.codigollm.es/

In [182]:
import sys

# Autenticación adicional para Google Colab
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

# @param {type:"string"}
PROJECT_ID = "tu-project-id"  # Sustituye con tu Project ID
# @param {type:"string"}
LOCATION = "tu-region-id"  # Elige tu región

if "google.colab" in sys.modules:
    import vertexai
    vertexai.init(project=PROJECT_ID, location=LOCATION)

# import os
# print(os.environ)

import vertexai

from google.oauth2 import service_account
CREDENTIALS = service_account.Credentials.from_service_account_file("ruta-a-tu-credentials.json")

vertexai.init(project=PROJECT_ID, credentials=CREDENTIALS)

In [237]:
import requests
from bs4 import BeautifulSoup
import html2text

def scrape_website(url):
    try:
        # Send an HTTP request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            return html2text.html2text(response.text)

        else:
            print(f"Failed to retrieve content. Status code: {response.status_code}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [238]:
url = "https://www.tiendeo.com/Tiendas/cerdanyola-del-valles/juguetes-y-bebes"

google_search_content = scrape_website(url)
google_search_content

'[![Tiendeo official website](https://static-webapp.tiendeo.com/assets/tiendeo-\nwhite.svg)](/cerdanyola-del-valles)\n\nCancelar\n\nBuscar\n\nMi Tiendeo\n\n  * [Destacados](/cerdanyola-del-valles)\n  * [Hiper-Supermercados](/cerdanyola-del-valles/supermercados)\n  * [Hogar y Muebles](/cerdanyola-del-valles/hogar-y-muebles)\n  * [Jardín y Bricolaje](/cerdanyola-del-valles/jardin-y-bricolaje)\n  * [Navidad](/cerdanyola-del-valles/navidad)\n  * [Ropa, Zapatos y Complementos](/cerdanyola-del-valles/ropa-zapatos-y-complementos)\n  * [Informática y Electrónica](/cerdanyola-del-valles/informatica-y-electronica)\n  * [Juguetes y Bebés](/cerdanyola-del-valles/juguetes-y-bebes)\n  * [Coches, Motos y Recambios](/cerdanyola-del-valles/coches-motos-y-recambios)\n  * [Perfumerías y Belleza](/cerdanyola-del-valles/perfumerias-y-belleza)\n  * [Viajes](/cerdanyola-del-valles/viajes)\n  * [Restauración](/cerdanyola-del-valles/restauracion)\n  * [Salud y Ópticas](/cerdanyola-del-valles/salud)\n  * [Ocio]

Si quieres ver un ejemplo más avanzado, mira este artículo (https://dev.to/albertgilopez/buqueda-en-tiempo-real-automatizada-con-google-maps-y-selenium-21h7) de “Búqueda en tiempo real automatizada con Google Maps y Selenium y este artículo (https://dev.to/albertgilopez/web-scraping-para-extraer-informacion-utilizando-requests-y-beautifulsoup-en-python-mmm) de “Web scraping para extraer información utilizando requests y BeautifulSoup en Python” en dev.to.

In [239]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def google_search_formated_response(content, max_output_tokens=7815):
  model = GenerativeModel("gemini-pro")

  schema = """
  {
    "places": [
      {
        "name": "",
        "rating": <<float>>,
        "price": "",
        "category": "",
        "address": "",
        "city": "",
        "state": "",
        "zip": "",
        "country": "",
        "phone": "",
        "website": ""
      }
    ]
  }
  """

  responses = model.generate_content(
    f"""Format the below response to the following JSON schema, only 1 store.

    Here's the content:

    {content}

    """,
        generation_config={
            "max_output_tokens": max_output_tokens,
            "temperature": 0,
            "top_p": 1
        },
      stream=True,
      )

  formated_response = []

  for response in responses:
      text = response.candidates[0].content.parts[0].text
      print(text)
      formated_response.append(text)

  return formated_response

formated_response = google_search_formated_response(google_search_content)

```json
{
  "stores": [
    {
      "
name": "Juguettos",
      "address": "Avenida
 Catalunya, 39",
      "city": "Cerdanyola del Vall\u00e8s",
      "state": null
,
      "zip": "08290",
      "country": "Spain",
      "phone": null,
      "hours
": "Closed",
      "distance": "0.53 km",
      "website": "https://www.tiendeo.com/Tiendas/cerdanyola-del-valles/juguet
tos-avenida-catalunya/199104",
      "logo": "https://static1.tiendeo.com/upload_negocio/negocio_1699/logo2.
png"
    }
  ]
}
```


In [240]:
formated_response

['```json\n{\n  "stores": [\n    {\n      "',
 'name": "Juguettos",\n      "address": "Avenida',
 ' Catalunya, 39",\n      "city": "Cerdanyola del Vall\\u00e8s",\n      "state": null',
 ',\n      "zip": "08290",\n      "country": "Spain",\n      "phone": null,\n      "hours',
 '": "Closed",\n      "distance": "0.53 km",\n      "website": "https://www.tiendeo.com/Tiendas/cerdanyola-del-valles/juguet',
 'tos-avenida-catalunya/199104",\n      "logo": "https://static1.tiendeo.com/upload_negocio/negocio_1699/logo2.',
 'png"\n    }\n  ]\n}\n```']

Ver también: Gemini AI: Extracción de datos estructurados con Gemini Pro Vision y Pydantic:

https://medium.com/@jddam/extracci%C3%B3n-de-datos-estructurados-con-gemini-pro-vision-y-pydantic-345577edb344
